In [2]:
import pyarrow # must occur prior to ray import
import ray
from ray import tune
from ray.tune import ExperimentAnalysis
from ray.tune.search.hyperopt import HyperOptSearch
import datetime
import numpy as np
import pandas as pd
import random
import seaborn as sns; sns.set()
from collections import Counter
from datasets import load_from_disk
from scipy.stats import ranksums
from sklearn.metrics import accuracy_score, f1_score
from transformers import BertForSequenceClassification
from transformers import Trainer
from transformers.training_args import TrainingArguments

from geneformer import DataCollatorForCellClassification
from geneformer import TranscriptomeTokenizer
import anndata as ad
import pandas as pd

2023-11-16 21:31:43,184	INFO util.py:159 -- Outdated packages:
  ipywidgets==7.7.0 found, needs ipywidgets>=8
Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.
2023-11-16 21:31:43,598	INFO util.py:159 -- Outdated packages:
  ipywidgets==7.7.0 found, needs ipywidgets>=8
Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


In [3]:
train_dataset=load_from_disk("/root/autodl-tmp/B_cell.dataset")

In [4]:
num_proc=30
train_dataset

Dataset({
    features: ['input_ids', 'cell_type', 'condition', 'length'],
    num_rows: 381
})

In [5]:
conditions = train_dataset["condition"]
target_names = list(set(conditions))

In [6]:
target_names

['Ctrl', 'AF']

In [7]:
target_name_id_dict = dict(zip(target_names,[i for i in range(len(target_names))]))

In [8]:
target_name_id_dict

{'Ctrl': 0, 'AF': 1}

In [9]:
trainset_v3 = train_dataset.rename_column("condition","label")

In [10]:
trainset_v3

Dataset({
    features: ['input_ids', 'cell_type', 'label', 'length'],
    num_rows: 381
})

In [11]:
def classes_to_ids(example):
    example["label"] = target_name_id_dict[example["label"]]
    return example

trainset_v4 = trainset_v3.map(classes_to_ids, num_proc=num_proc)

Map (num_proc=30):   0%|          | 0/381 [00:00<?, ? examples/s]

/root/miniconda3/lib/python3.8/site-packages/datasets/table.py:1421: FutureWarning: promote has been superseded by mode='default'.
  table = cls._concat_blocks(blocks, axis=0)


In [12]:
indiv_list = trainset_v4["input_ids"]

In [13]:
len(indiv_list)

381

In [14]:
random.seed(42)
train_indiv = random.sample(indiv_list,round(0.7*len(indiv_list)))
eval_indiv = [indiv for indiv in indiv_list if indiv not in train_indiv]
valid_indiv = random.sample(eval_indiv,round(0.5*len(eval_indiv)))
test_indiv = [indiv for indiv in eval_indiv if indiv not in valid_indiv]

In [15]:
def if_train(example):
    return example["input_ids"] in train_indiv

classifier_trainset = trainset_v4.filter(if_train,num_proc=num_proc).shuffle(seed=42)

Filter (num_proc=30):   0%|          | 0/381 [00:00<?, ? examples/s]

/root/miniconda3/lib/python3.8/site-packages/datasets/table.py:1387: FutureWarning: promote has been superseded by mode='default'.
  return cls._concat_blocks(pa_tables_to_concat_vertically, axis=0)
/root/miniconda3/lib/python3.8/site-packages/datasets/table.py:1387: FutureWarning: promote has been superseded by mode='default'.
  return cls._concat_blocks(pa_tables_to_concat_vertically, axis=0)
/root/miniconda3/lib/python3.8/site-packages/datasets/table.py:1387: FutureWarning: promote has been superseded by mode='default'.
  return cls._concat_blocks(pa_tables_to_concat_vertically, axis=0)
/root/miniconda3/lib/python3.8/site-packages/datasets/table.py:1387: FutureWarning: promote has been superseded by mode='default'.
  return cls._concat_blocks(pa_tables_to_concat_vertically, axis=0)
/root/miniconda3/lib/python3.8/site-packages/datasets/table.py:1387: FutureWarning: promote has been superseded by mode='default'.
  return cls._concat_blocks(pa_tables_to_concat_vertically, axis=0)
/root

In [16]:
def if_valid(example):
    return example["input_ids"] in valid_indiv

classifier_validset = trainset_v4.filter(if_valid,num_proc=num_proc).shuffle(seed=42)

Filter (num_proc=30):   0%|          | 0/381 [00:00<?, ? examples/s]

/root/miniconda3/lib/python3.8/site-packages/datasets/table.py:1387: FutureWarning: promote has been superseded by mode='default'.
  return cls._concat_blocks(pa_tables_to_concat_vertically, axis=0)
/root/miniconda3/lib/python3.8/site-packages/datasets/table.py:1387: FutureWarning: promote has been superseded by mode='default'.
  return cls._concat_blocks(pa_tables_to_concat_vertically, axis=0)
/root/miniconda3/lib/python3.8/site-packages/datasets/table.py:1387: FutureWarning: promote has been superseded by mode='default'.
  return cls._concat_blocks(pa_tables_to_concat_vertically, axis=0)
/root/miniconda3/lib/python3.8/site-packages/datasets/table.py:1387: FutureWarning: promote has been superseded by mode='default'.
  return cls._concat_blocks(pa_tables_to_concat_vertically, axis=0)
/root/miniconda3/lib/python3.8/site-packages/datasets/table.py:1387: FutureWarning: promote has been superseded by mode='default'.
  return cls._concat_blocks(pa_tables_to_concat_vertically, axis=0)
/root

In [17]:
def if_test(example):
    return example["input_ids"] in test_indiv

classifier_testdset = trainset_v4.filter(if_test,num_proc=num_proc).shuffle(seed=42)

Filter (num_proc=30):   0%|          | 0/381 [00:00<?, ? examples/s]

/root/miniconda3/lib/python3.8/site-packages/datasets/table.py:1387: FutureWarning: promote has been superseded by mode='default'.
  return cls._concat_blocks(pa_tables_to_concat_vertically, axis=0)
/root/miniconda3/lib/python3.8/site-packages/datasets/table.py:1387: FutureWarning: promote has been superseded by mode='default'.
  return cls._concat_blocks(pa_tables_to_concat_vertically, axis=0)
/root/miniconda3/lib/python3.8/site-packages/datasets/table.py:1387: FutureWarning: promote has been superseded by mode='default'.
  return cls._concat_blocks(pa_tables_to_concat_vertically, axis=0)
/root/miniconda3/lib/python3.8/site-packages/datasets/table.py:1387: FutureWarning: promote has been superseded by mode='default'.
  return cls._concat_blocks(pa_tables_to_concat_vertically, axis=0)
/root/miniconda3/lib/python3.8/site-packages/datasets/table.py:1387: FutureWarning: promote has been superseded by mode='default'.
  return cls._concat_blocks(pa_tables_to_concat_vertically, axis=0)
/root

In [21]:
current_date = datetime.datetime.now()
datestamp = f"{str(current_date.year)[-2:]}{current_date.month:02d}{current_date.day:02d}"
output_dir = f"/root/autodl-tmp/B_cell/model/{datestamp}_geneformer_DiseaseClassifier/"

In [22]:
import os
saved_model_test = os.path.join(output_dir, f"pytorch_model.bin")
if os.path.isfile(saved_model_test) == True:
    raise Exception("Model already saved to this directory.")

In [23]:
# make output directory
import subprocess
subprocess.call(f'mkdir {output_dir}', shell=True)

0

In [24]:
# set training parameters
# how many pretrained layers to freeze
freeze_layers = 2
# batch size for training and eval
geneformer_batch_size = 12
# number of epochs
epochs = 2
# logging steps
logging_steps = round(len(classifier_trainset)/geneformer_batch_size/10)

In [25]:
def model_init():
    model = BertForSequenceClassification.from_pretrained("/root/autodl-tmp/Geneformer/",
                                                          num_labels=len(target_names),
                                                          output_attentions = False,
                                                          output_hidden_states = False)
    if freeze_layers is not None:
        modules_to_freeze = model.bert.encoder.layer[:freeze_layers]
        for module in modules_to_freeze:
            for param in module.parameters():
                param.requires_grad = False

    return model

In [26]:
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    # calculate accuracy and macro f1 using sklearn's function
    acc = accuracy_score(labels, preds)
    macro_f1 = f1_score(labels, preds, average='macro')
    return {
      'accuracy': acc,
      'macro_f1': macro_f1
    }

In [29]:
training_args = {
    "do_train": True,
    "do_eval": True,
    "evaluation_strategy": "steps",
    "eval_steps": logging_steps,
    "logging_steps": logging_steps,
    "group_by_length": True,
    "length_column_name": "length",
    "disable_tqdm": True,
    "skip_memory_metrics": True, # memory tracker causes errors in raytune
    "per_device_train_batch_size": geneformer_batch_size,
    "per_device_eval_batch_size": geneformer_batch_size,
    "num_train_epochs": epochs,
    "load_best_model_at_end": True,
    "output_dir": output_dir,
    "save_steps":2,
}

In [30]:
training_args_init = TrainingArguments(**training_args)

In [31]:
trainer = Trainer(
    model_init=model_init,
    args=training_args_init,
    data_collator=DataCollatorForCellClassification(),
    train_dataset=classifier_trainset,
    eval_dataset=classifier_validset,
    compute_metrics=compute_metrics,
)

Detected kernel version 5.4.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at /root/autodl-tmp/Geneformer/ and are newly initialized: ['classifier.weight', 'bert.pooler.dense.weight', 'bert.pooler.dense.bias', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [32]:
ray_config = {
    "num_train_epochs": tune.choice([epochs]),
    "learning_rate": tune.loguniform(1e-6, 1e-3),
    "weight_decay": tune.uniform(0.0, 0.3),
    "lr_scheduler_type": tune.choice(["linear","cosine","polynomial"]),
    "warmup_steps": tune.uniform(100, 2000),
    "seed": tune.uniform(0,100),
    "per_device_train_batch_size": tune.choice([geneformer_batch_size])
}

In [33]:
hyperopt_search = HyperOptSearch(
    metric="eval_accuracy", mode="max")

In [34]:
# optimize hyperparameters
trainer.hyperparameter_search(
    direction="maximize",
    backend="ray",
    resources_per_trial={"cpu":8,"gpu":1},
    hp_space=lambda _: ray_config,
    search_alg=hyperopt_search,
    n_trials=100, # number of trials
    progress_reporter=tune.CLIReporter(max_report_frequency=600,
                                                   sort_by_metric=True,
                                                   max_progress_rows=100,
                                                   mode="max",
                                                   metric="eval_accuracy",
                                                   metric_columns=["loss", "eval_loss", "eval_accuracy","macro_f1"])
)

DeprecationWarning: Accepting a `checkpoint_dir` argument in your training function is deprecated.
Please use `ray.train.get_checkpoint()` to access your checkpoint as a
`ray.train.Checkpoint` object instead. See below for an example:

Before
------

from ray import tune

def train_fn(config, checkpoint_dir=None):
    if checkpoint_dir:
        torch.load(os.path.join(checkpoint_dir, "checkpoint.pt"))
    ...

tuner = tune.Tuner(train_fn)
tuner.fit()

After
-----

from ray import train, tune

def train_fn(config):
    checkpoint: train.Checkpoint = train.get_checkpoint()
    if checkpoint:
        with checkpoint.as_directory() as checkpoint_dir:
            torch.load(os.path.join(checkpoint_dir, "checkpoint.pt"))
    ...

tuner = tune.Tuner(train_fn)
tuner.fit()

In [35]:
trainer.train()

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at /root/autodl-tmp/Geneformer/ and are newly initialized: ['classifier.weight', 'bert.pooler.dense.weight', 'bert.pooler.dense.bias', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/root/miniconda3/lib/python3.8/site-packages/geneformer/collator_for_classification.py:581: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  batch = {k: torch.tensor(v, dtype=torch.int64) for k, v in batch.items()}


{'loss': 0.6848, 'learning_rate': 4.782608695652174e-05, 'epoch': 0.09}
{'eval_loss': 0.6521574854850769, 'eval_accuracy': 0.7368421052631579, 'eval_macro_f1': 0.4242424242424242, 'eval_runtime': 0.3812, 'eval_samples_per_second': 149.514, 'eval_steps_per_second': 13.115, 'epoch': 0.09}


/root/miniconda3/lib/python3.8/site-packages/geneformer/collator_for_classification.py:581: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  batch = {k: torch.tensor(v, dtype=torch.int64) for k, v in batch.items()}


{'loss': 0.7019, 'learning_rate': 4.565217391304348e-05, 'epoch': 0.17}
{'eval_loss': 0.6317639350891113, 'eval_accuracy': 0.7368421052631579, 'eval_macro_f1': 0.4242424242424242, 'eval_runtime': 0.3857, 'eval_samples_per_second': 147.795, 'eval_steps_per_second': 12.964, 'epoch': 0.17}


/root/miniconda3/lib/python3.8/site-packages/geneformer/collator_for_classification.py:581: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  batch = {k: torch.tensor(v, dtype=torch.int64) for k, v in batch.items()}


...

/root/miniconda3/lib/python3.8/site-packages/geneformer/collator_for_classification.py:581: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  batch = {k: torch.tensor(v, dtype=torch.int64) for k, v in batch.items()}


{'loss': 0.5461, 'learning_rate': 2.173913043478261e-06, 'epoch': 1.91}
{'eval_loss': 0.5589110851287842, 'eval_accuracy': 0.7368421052631579, 'eval_macro_f1': 0.4242424242424242, 'eval_runtime': 0.3841, 'eval_samples_per_second': 148.393, 'eval_steps_per_second': 13.017, 'epoch': 1.91}


/root/miniconda3/lib/python3.8/site-packages/geneformer/collator_for_classification.py:581: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  batch = {k: torch.tensor(v, dtype=torch.int64) for k, v in batch.items()}


{'loss': 0.9017, 'learning_rate': 0.0, 'epoch': 2.0}
{'eval_loss': 0.5588369369506836, 'eval_accuracy': 0.7368421052631579, 'eval_macro_f1': 0.4242424242424242, 'eval_runtime': 0.3847, 'eval_samples_per_second': 148.174, 'eval_steps_per_second': 12.998, 'epoch': 2.0}
{'train_runtime': 20.6609, 'train_samples_per_second': 25.846, 'train_steps_per_second': 2.226, 'train_loss': 0.637907950774483, 'epoch': 2.0}


TrainOutput(global_step=46, training_loss=0.637907950774483, metrics={'train_runtime': 20.6609, 'train_samples_per_second': 25.846, 'train_steps_per_second': 2.226, 'train_loss': 0.637907950774483, 'epoch': 2.0})

In [36]:
predictions = trainer.predict(classifier_testdset)

/root/miniconda3/lib/python3.8/site-packages/geneformer/collator_for_classification.py:581: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  batch = {k: torch.tensor(v, dtype=torch.int64) for k, v in batch.items()}


In [37]:
import pickle
with open(f"{output_dir}predictions.pickle", "wb") as fp:
        pickle.dump(predictions, fp)
trainer.save_metrics("eval",predictions.metrics)
trainer.save_model(output_dir)